# US Census Data Science

1. Compare home ownership rates across states - looks at % owner-occupied housing units with a mortgage by state
2. Analyze median household income differences across states - provides state level median household income estimates
3. Assess housing affordability by state - can look at median home values compared to median income to evaluate affordability
4. Identify most/least affordable states for housing - rank states by typical housing costs as a percentage of household income
5. Evaluate real estate tax burdens by state - median real estate taxes paid compared to median income
6. Assess mortgage debt levels - distribution of housing units with a mortgage across loan size buckets
7. Identify states with highest mortgage debt - rank by % of units with mortgages and median mortgage amount
8. Analyze trends in home values over time - compare median home values year-over-year from census data
9. Segment states by predominant housing type - cluster analysis on housing structure types by state
10. Predict home values based on housing characteristics - build a regression model to estimate home values
11. Forecast mortgage interest rates - time series analysis and forecasting of mortgage rates
12. Identify associations between demographics and home ownership - correlation analysis between ownership and factors like income, age etc.

In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import re

%matplotlib inline
init_notebook_mode(connected=True) 

In [3]:
data_2022 = pd.read_csv('../app/data/AverageHouseHoldSize2022.csv', index_col=0)
data_2022= data_2022.transpose()
data_2022 = data_2022.dropna(axis=1)

data_2022.head()

State,Total:,Owner occupied,Renter occupied
Alabama,2.45,2.54,2.22
Alaska,2.56,2.69,2.31
Arizona,2.53,2.59,2.39
Arkansas,2.44,2.56,2.20
California,2.82,2.94,2.66


In [4]:
data_2012 = pd.read_csv('../app/data/AverageHouseHoldSize2012.csv', index_col=0)
data_2012= data_2012.transpose()
data_2012 = data_2012.dropna(axis=1)

data_2012.head()

State,Total:,Owner occupied,Renter occupied
Alabama,2.55,2.58,2.49
Alaska,2.80,2.91,2.62
Arizona,2.68,2.66,2.70
Arkansas,2.51,2.53,2.46
California,2.97,3.00,2.93


In [5]:
state_abbreviations = {
'Alabama': 'AL',
'Alaska': 'AK',
'Arizona': 'AZ',
'Arkansas': 'AR',
'California': 'CA',
'Colorado': 'CO',
'Connecticut': 'CT',
'Delaware': 'DE',
'District of Columbia': 'DC',
'Florida': 'FL',
'Georgia': 'GA',
'Hawaii': 'HI',
'Idaho': 'ID',
'Illinois': 'IL',
'Indiana': 'IN',
'Iowa': 'IA',
'Kansas': 'KS',
'Kentucky': 'KY',
'Louisiana': 'LA',
'Maine': 'ME',
'Maryland': 'MD',
'Massachusetts': 'MA',
'Michigan': 'MI',
'Minnesota': 'MN',
'Mississippi': 'MS',
'Missouri': 'MO',
'Montana': 'MT',
'Nebraska': 'NE',
'Nevada': 'NV',
'New Hampshire': 'NH',
'New Jersey': 'NJ',
'New Mexico': 'NM',
'New York': 'NY',
'North Carolina': 'NC',
'North Dakota': 'ND',
'Ohio': 'OH',
'Oklahoma': 'OK',
'Oregon': 'OR',
'Pennsylvania': 'PA',
'Rhode Island': 'RI',
'South Carolina': 'SC',
'South Dakota': 'SD',
'Tennessee': 'TN',
'Texas': 'TX',
'Utah': 'UT',
'Vermont': 'VT',
'Virginia': 'VA',
'Washington': 'WA',
'West Virginia': 'WV',
'Wisconsin': 'WI',
'Wyoming': 'WY',
'Puerto Rico': 'PR'
}

In [117]:
def dataframe_cleanup(df):
    
    #Clean up dataframe columns
    dict_data = df.to_dict()

    new_dict = {'State': [], 'Owner_occupied': [], 'Renter_occupied': []}
    for k, v in dict_data.items():
        if k=='Owner occupied':
            new_dict['State'].extend(list(v.keys())),
            new_dict['Owner_occupied'].extend(v.values())

        if k=='Renter occupied':
            new_dict['Renter_occupied'].extend(v.values())


    #Add state abbreviations to the dataframe column
    df = pd.DataFrame(new_dict, columns=['State', 'Owner_occupied','Renter_occupied'])
    df['Code'] = df['State'].map(state_abbreviations)
            
    return df

In [118]:
df_2022 = dataframe_cleanup(data_2022)

In [119]:
df_2022.head()

,State,Owner_occupied,Renter_occupied,Code
0,Alabama,2.54,2.22,AL
1,Alaska,2.69,2.31,AK
2,Arizona,2.59,2.39,AZ
3,Arkansas,2.56,2.20,AR
4,California,2.94,2.66,CA


In [137]:
#Make choropleth map

def make_map(df, data_year: str):

    data = dict(type = 'choropleth',
                colorscale = 'Viridis_r',
                locations = df['Code'],
                locationmode = 'USA-states',
                z=df['Owner_occupied'],
                colorbar = {'title':'Average Number of People'})

    layout = dict(
        title = {'text': f'{data_year} US Average Household Size by Home Owner', 'x':0.5, 'xanchor': 'center'},
        geo = dict(scope = 'usa'),
        autosize = True,
        width = 800,
        height = 600,
    )

    choromap_own = go.Figure(data = [data],layout = layout)

    data_rent = dict(type = 'choropleth',
                colorscale = 'Portland',
                locations = df['Code'],
                locationmode = 'USA-states',
                z=df['Renter_occupied'],
                colorbar = {'title':'Average Number of People'})

    layout = dict(
        title = {'text': f'{data_year} US Average Household Size by Renter', 'x':0.5, 'xanchor': 'center'},
        geo = dict(scope = 'usa'),
        autosize = True,
        width = 800,
        height = 600,
    )

    choromap = go.Figure(data = [data_rent],layout = layout)

    iplot(choromap_own)
    iplot(choromap)

In [138]:
make_map(df_2022, '2022')

In [132]:
df_2012 = dataframe_cleanup(data_2012)

In [12]:
df_2012.head()

,State,Owner_occupied,Renter_occupied,Code
0,Alabama,2.58,2.49,AL
1,Alaska,2.91,2.62,AK
2,Arizona,2.66,2.70,AZ
3,Arkansas,2.53,2.46,AR
4,California,3.00,2.93,CA


In [13]:
make_map(df_2012, '2012')

In [14]:
def make_bar_graph(df, state: str, data_year: str):
    #grab the state for comparison
    target_state = df[df['State']==state].iloc[0]
    #conver series to dataframe
    target_state = target_state.to_frame().T

    # Melt DataFrame to have one row per category
    df_melt = target_state.melt(id_vars=['State', 'Code'], value_vars=['Owner_occupied', 'Renter_occupied'], var_name='Category', value_name='Avg Household Size')

    # Create the bar chart
    fig = px.bar(df_melt, x='Category', y='Avg Household Size', color='Category', title=f'{data_year} Comparison of Owner Occupied vs Renter Occupied for {state}')
    fig.show()


In [15]:
make_bar_graph(df_2012, 'California', '2012')

# Population Pyramid

In [16]:
def clean_headers(val):
    if isinstance(val, str):
        val = val.split("!!")[0]
        return val
    else:
        return val

In [17]:
data_pop = pd.read_csv('../app/data/SexByAge2022.csv', index_col=0)
data_pop = data_pop.rename(columns=clean_headers)
data_pop.head()

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Puerto Rico
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Total:,"5,074,296","733,583","7,359,197","3,045,637","39,029,342","5,839,926","3,626,205","1,018,396","671,803","22,244,823",...,"7,051,339","30,029,572","3,380,800","647,064","8,683,619","7,785,786","1,775,156","5,892,539","581,381","3,221,789"
Male:,"2,461,248","385,667","3,678,381","1,504,488","19,536,425","2,960,896","1,776,689","494,657","319,763","10,953,468",...,"3,462,618","15,021,141","1,716,117","320,698","4,299,314","3,930,411","882,101","2,958,771","297,855","1,528,789"
Under 5 years,"146,169","23,043","201,423","90,239","1,081,904","154,565","91,513","27,456","20,038","563,703",...,"206,317","963,076","116,718","13,651","246,632","213,623","43,466","159,162","15,051","53,485"
5 to 9 years,"158,767","25,916","221,769","98,535","1,165,348","166,104","97,762","29,704","19,567","598,155",...,"217,274","1,061,578","132,973","15,723","265,004","242,230","49,137","173,416","20,443","68,924"
10 to 14 years,"164,578","26,492","238,114","103,801","1,313,295","185,183","113,665","30,008","15,307","642,573",...,"226,099","1,105,937","142,439","17,022","278,627","240,832","52,066","190,753","19,356","86,034"


In [18]:
male_data = data_pop.iloc[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]

In [19]:
female_data = data_pop.iloc[[26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]]

In [20]:
male_data.reset_index()

,Label (Grouping),Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Puerto Rico
0,Under 5 years,"146,169","23,043","201,423","90,239","1,081,904","154,565","91,513","27,456","20,038",...,"206,317","963,076","116,718","13,651","246,632","213,623","43,466","159,162","15,051","53,485"
1,5 to 9 years,"158,767","25,916","221,769","98,535","1,165,348","166,104","97,762","29,704","19,567",...,"217,274","1,061,578","132,973","15,723","265,004","242,230","49,137","173,416","20,443","68,924"
2,10 to 14 years,"164,578","26,492","238,114","103,801","1,313,295","185,183","113,665","30,008","15,307",...,"226,099","1,105,937","142,439","17,022","278,627","240,832","52,066","190,753","19,356","86,034"
3,15 to 17 years,"97,834","13,810","149,233","64,589","794,712","112,338","71,153","18,616","8,130",...,"137,910","671,756","87,820","11,838","168,199","147,643","32,895","118,151","12,413","61,229"
4,18 and 19 years,"70,450","8,971","101,519","42,479","520,164","78,156","54,600","13,233","8,771",...,"89,949","440,629","54,370","10,097","122,941","87,181","24,698","81,143","7,913","42,337"
5,20 years,"42,597","5,067","51,011","24,372","281,743","40,402","22,434","6,284","4,020",...,"49,867","213,017","23,224","6,504","63,821","46,292","13,007","44,375","3,821","23,156"
6,21 years,"34,623","4,903","57,654","21,652","276,546","43,388","21,585","6,161","4,981",...,"43,860","220,172","30,794","4,060","65,992","51,981","13,713","38,060","4,966","22,538"
7,22 to 24 years,"97,373","19,628","160,086","63,841","799,942","124,164","77,844","18,209","12,712",...,"146,283","669,431","100,579","12,110","178,124","166,251","29,688","125,781","10,340","63,663"
8,25 to 29 years,"159,971","30,938","266,176","92,694","1,420,961","232,970","114,646","30,555","30,920",...,"235,006","1,084,997","131,378","19,379","283,753","293,952","55,461","188,178","18,737","106,264"
9,30 to 34 years,"162,451","31,914","264,266","100,113","1,544,190","247,358","114,723","34,015","36,549",...,"244,484","1,125,087","123,560","20,704","306,840","319,143","54,866","188,427","20,342","97,878"


In [21]:
female_data.reset_index()

,Label (Grouping),Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Puerto Rico
0,Under 5 years,"137,895","23,454","191,990","87,526","1,036,482","150,498","86,940","26,602","18,997",...,"195,898","918,642","111,746","14,224","235,050","208,099","44,003","150,082","15,393","45,947"
1,5 to 9 years,"152,825","23,721","216,589","97,918","1,120,088","158,368","98,043","27,814","17,701",...,"204,848","1,009,224","120,860","14,882","242,489","221,805","47,080","160,270","16,828","64,271"
2,10 to 14 years,"153,945","24,686","225,438","96,588","1,236,877","175,695","103,912","29,897","16,251",...,"213,714","1,067,195","135,681","17,103","265,548","232,574","50,753","178,348","19,345","84,723"
3,15 to 17 years,"99,015","14,623","143,047","56,636","750,300","108,344","67,436","18,130","8,253",...,"133,073","642,255","82,356","9,291","160,538","137,221","32,143","111,598","10,774","53,723"
4,18 and 19 years,"72,768","7,390","93,935","39,796","503,612","70,976","52,486","12,914","11,514",...,"86,016","406,095","53,410","11,785","117,524","85,015","22,024","74,110","7,187","39,907"
5,20 years,"36,614","5,676","45,508","18,719","253,659","37,189","26,602","8,114","3,450",...,"45,484","200,068","24,460","6,010","55,771","38,536","10,967","40,744","5,333","18,342"
6,21 years,"32,968","3,438","52,940","20,136","261,325","36,166","22,700","5,290","4,823",...,"46,153","203,110","31,869","4,002","61,339","45,134","12,461","40,692","2,761","27,173"
7,22 to 24 years,"100,951","13,547","155,285","61,186","766,572","114,386","72,342","15,923","18,057",...,"140,484","640,004","88,589","11,119","177,238","151,160","34,489","117,354","9,661","61,899"
8,25 to 29 years,"155,194","23,138","241,073","94,582","1,334,198","212,220","106,285","31,035","39,184",...,"236,274","1,039,623","121,672","18,376","272,475","267,525","46,762","177,983","16,234","104,293"
9,30 to 34 years,"174,536","27,910","247,176","98,488","1,445,110","223,394","114,856","34,958","38,600",...,"248,424","1,079,650","118,166","19,056","293,554","295,677","50,003","179,141","17,121","97,399"


In [22]:
def cleanup_dataframe(df, gender: str):

    dict_data = df.to_dict()

    #Clean up label column for white spaces and convert population size to integers
    cleaned_dict = {state: {key.strip(): int(float(value.replace(',',''))) for key, value in data.items()} for state, data in dict_data.items()}

    final_dict = {'State': [], 'Age Group': [], gender: []}

    #Clean up age_group labels
    age_groups = ['0 - 4','5 - 9','10 - 14','15 - 19' ,'20 - 24','25 - 29','30 - 34', '35 - 39','40 - 44', '45 - 49','50 - 54','55 - 59','60 - 64','65 - 69','70 - 74','75 - 79', '80 - 84', '85 +']

    for state, data in cleaned_dict.items():
        gender_data = [data['Under 5 years'], data['5 to 9 years'], 
                data['10 to 14 years'],(data['15 to 17 years'] + data['18 and 19 years']),
                (data['20 years'] + data['21 years'] + data['22 to 24 years']), data['25 to 29 years'],
                data['30 to 34 years'],data['35 to 39 years'],
                data['40 to 44 years'], data['45 to 49 years'],
                data['50 to 54 years'], data['55 to 59 years'],
                (data['60 and 61 years'] + data['62 to 64 years']),(data['65 and 66 years'] + data['67 to 69 years']),
                data['70 to 74 years'],data['75 to 79 years'],
                data['80 to 84 years'],data['85 years and over']]
        
        for age_group, g in zip(age_groups, gender_data):
            final_dict['State'].append(state)
            final_dict['Age Group'].append(age_group)
            final_dict[gender].append(g)

    df = pd.DataFrame(final_dict)

    return df

In [23]:
df_male_2022 = cleanup_dataframe(male_data,'Males')
df_female_2022 = cleanup_dataframe(male_data,'Females')


In [24]:
df_male_2022.head()

,State,Age Group,Males
0,Alabama,0 - 4,146169
1,Alabama,5 - 9,158767
2,Alabama,10 - 14,164578
3,Alabama,15 - 19,168284
4,Alabama,20 - 24,174593


In [25]:
def graph_pyramid(df_male,df_female,state: str, data_year: str):

    df = pd.merge(df_male, df_female)
    #Add State Code
    df['Code'] = df['State'].map(state_abbreviations)

    target_state = df.loc[df['State']==state]
    
    y = target_state['Age Group']
    x1 = target_state['Males']
    x2 = target_state['Females'] * -1
    state = target_state['State'].iloc[0]
    state_code = target_state['Code'].iloc[0]

    #Create instance for chart figure
    fig = go.Figure()

    #Add Trace to Figure
    fig.add_trace(go.Bar(
        y = y,
        x = x1,
        name = 'Males',
        orientation='h'
    ))

    #Add Trace to figure
    fig.add_trace(go.Bar(
        y = y,
        x = x2,
        name = 'Females',
        orientation='h'
    ))

    # Find the minimum and maximum values in x1 and x2
    min_val = min(min(x1), min(x2))
    max_val = max(max(x1), max(x2))

    # Create a range of values for the x-axis ticks
    tickvals = list(range(min_val, max_val, int((max_val - min_val) / 5)))

    def human_format(num):
        magnitude = 0
        while abs(num) >= 1000:
            magnitude += 1
            num /= 1000.0
        return '%.1f%s' % (num, ['', 'K', 'M', 'G', 'T', 'P'][magnitude])

    #Update the Title_suffix
    max_val = max(max(x1), max(x2))
    if abs(max_val) >= 10**6:
        title_suffix = 'in Millions'
    elif abs(max_val) >= 10**3:
        title_suffix = 'in Thousands'
    else:
        title_suffix = ''

    #Update Figure Layout
    fig.update_layout(
        template = 'plotly_white',
        title = f'Age Pyramid {state} {data_year}',
        title_font_size = 24,
        barmode = 'relative',
        bargap =0.0,
        bargroupgap =0,
        height = 700,
        width = 1400,
        xaxis= dict(
            tickvals=tickvals,
            ticktext=[human_format(abs(i)) for i in tickvals],
            title = 'Populations ' + title_suffix,
            title_font_size=14
        )
    )

    #Plot Figure
    fig.show()




In [26]:
graph_pyramid(df_male_2022,df_female_2022,'California', '2022')

In [28]:
data_pop = pd.read_csv('../Unused_Data/SexByAge2012.csv', index_col=0)
data_pop = data_pop.rename(columns=clean_headers)
data_pop.head()

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Puerto Rico
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Total:,"4,822,023","731,449","6,553,255","2,949,131","38,041,430","5,187,582","3,590,347","917,092","632,323","19,317,568",...,"6,456,243","26,059,203","2,855,287","626,011","8,185,867","6,897,012","1,855,413","5,726,398","576,412","3,667,084"
Male:,"2,336,291","381,094","3,260,687","1,449,265","18,917,773","2,601,370","1,749,806","444,691","299,036","9,441,001",...,"3,144,756","12,945,218","1,435,028","309,184","4,015,045","3,441,429","911,681","2,842,737","292,736","1,757,273"
Under 5 years,"154,902","27,593","223,705","99,007","1,296,287","172,562","99,546","28,446","19,889","545,488",...,"203,801","985,806","132,481","16,283","256,456","226,381","53,538","179,576","19,401","108,250"
5 to 9 years,"158,765","27,033","236,024","105,750","1,296,812","187,549","111,455","28,624","15,317","568,554",...,"213,254","1,000,422","128,285","16,964","268,185","222,732","52,269","186,241","18,842","122,716"
10 to 14 years,"163,731","26,425","230,633","103,149","1,306,123","168,057","119,297","29,963","11,832","569,454",...,"215,096","1,005,935","126,174","19,223","263,164","229,458","56,163","192,451","21,383","123,441"


In [29]:
male_data = data_pop.iloc[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]]

In [30]:
female_data = data_pop.iloc[[26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48]]

In [31]:
male_data.reset_index()

,Label (Grouping),Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Puerto Rico
0,Under 5 years,"154,902","27,593","223,705","99,007","1,296,287","172,562","99,546","28,446","19,889",...,"203,801","985,806","132,481","16,283","256,456","226,381","53,538","179,576","19,401","108,250"
1,5 to 9 years,"158,765","27,033","236,024","105,750","1,296,812","187,549","111,455","28,624","15,317",...,"213,254","1,000,422","128,285","16,964","268,185","222,732","52,269","186,241","18,842","122,716"
2,10 to 14 years,"163,731","26,425","230,633","103,149","1,306,123","168,057","119,297","29,963","11,832",...,"215,096","1,005,935","126,174","19,223","263,164","229,458","56,163","192,451","21,383","123,441"
3,15 to 17 years,"97,673","15,899","138,579","59,618","825,862","100,405","75,376","17,597","7,946",...,"130,443","575,339","68,160","11,685","157,048","134,936","33,200","117,112","10,748","83,047"
4,18 and 19 years,"73,658","12,008","93,580","41,556","569,900","77,113","55,962","13,530","10,100",...,"88,477","391,694","44,467","10,877","125,452","94,694","24,894","84,078","8,188","59,087"
5,20 years,"36,883","5,837","52,451","22,402","316,738","38,793","24,703","8,158","5,739",...,"51,395","217,442","20,720","6,229","64,960","50,186","15,781","42,976","4,961","30,186"
6,21 years,"40,183","7,258","51,576","23,326","318,160","39,785","25,448","7,982","5,384",...,"44,555","213,456","23,336","4,767","67,540","51,409","14,262","38,583","3,749","28,553"
7,22 to 24 years,"96,152","19,766","142,091","59,553","878,262","109,601","69,139","17,374","17,154",...,"129,822","568,091","75,973","12,597","166,614","149,146","33,618","115,917","13,808","77,792"
8,25 to 29 years,"148,583","29,302","228,596","94,436","1,442,237","196,863","110,732","29,294","34,059",...,"211,551","961,511","110,949","18,256","291,796","251,654","53,212","185,671","21,833","108,161"
9,30 to 34 years,"145,284","26,971","222,080","94,135","1,370,890","197,272","105,644","27,464","30,707",...,"206,272","939,879","115,308","16,928","281,999","242,790","56,970","188,327","20,646","113,251"


In [32]:
female_data.reset_index()

,Label (Grouping),Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Puerto Rico
0,Under 5 years,"146,906","26,341","214,067","92,649","1,240,523","164,433","93,524","27,739","19,113",...,"196,222","947,042","125,448","14,728","250,615","214,592","51,725","171,659","18,615","101,104"
1,5 to 9 years,"154,067","25,172","218,351","99,763","1,239,678","172,087","104,747","28,419","15,075",...,"204,091","962,135","124,265","15,940","248,048","206,463","53,452","184,000","19,774","109,395"
2,10 to 14 years,"157,592","24,487","227,456","94,845","1,245,997","170,502","116,104","27,153","12,042",...,"207,377","955,210","117,765","17,662","256,832","220,655","52,654","178,137","17,143","123,632"
3,15 to 17 years,"91,339","14,315","130,770","56,848","788,024","95,712","72,717","16,979","8,238",...,"123,933","549,286","64,610","11,404","154,284","129,812","31,219","107,574","10,704","77,776"
4,18 and 19 years,"69,110","7,820","87,001","40,105","535,046","69,187","53,246","14,294","13,531",...,"84,936","372,597","45,112","11,385","117,458","87,226","22,658","82,409","9,159","57,712"
5,20 years,"37,248","5,608","44,722","23,594","299,727","34,399","25,929","7,911","6,566",...,"48,331","199,744","25,344","4,122","59,925","47,378","15,049","41,581","4,105","27,114"
6,21 years,"39,243","5,844","51,581","22,324","289,211","36,106","22,923","6,567","5,753",...,"46,221","184,543","24,603","6,259","61,908","48,527","13,478","40,448","3,016","31,440"
7,22 to 24 years,"94,925","15,449","130,204","55,260","815,296","106,637","62,253","17,744","19,113",...,"131,816","540,082","67,209","10,938","162,612","137,558","35,252","109,486","11,874","73,702"
8,25 to 29 years,"157,065","28,378","213,084","96,970","1,342,594","181,379","106,989","29,398","41,099",...,"210,610","924,396","108,412","17,665","283,874","239,281","52,091","180,479","19,447","113,582"
9,30 to 34 years,"157,911","24,680","212,154","93,872","1,318,597","181,050","109,244","27,692","33,013",...,"215,016","925,729","110,445","17,451","280,943","236,881","55,778","180,003","18,153","121,670"


In [33]:
df_male_2012 = cleanup_dataframe(male_data,'Males')
df_female_2012 = cleanup_dataframe(male_data,'Females')

In [34]:
graph_pyramid(df_male_2012,df_female_2012,'California', '2012')

# Financial Mortgage Data

In [35]:
def clean_mortgage_headers(val):
    if isinstance(val, str):
        if 'Percent' in val:
            val = val.split("!!")[0]
            val = val + " Percent"
        else:
            val = val.split("!!")[0]
        return val
    else:
        return val

In [36]:
mortgage_data = pd.read_csv('../app/data/Financial_Mortgate_Data_2022.csv', index_col=0)
mortgage_data = mortgage_data.rename(columns=clean_mortgage_headers)
mortgage_data.head()

,Alabama,Alabama Percent,Alaska,Alaska Percent,Arizona,Arizona Percent,Arkansas,Arkansas Percent,California,California Percent,...,Washington,Washington Percent,West Virginia,West Virginia Percent,Wisconsin,Wisconsin Percent,Wyoming,Wyoming Percent,Puerto Rico,Puerto Rico Percent
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Owner-occupied housing units with a mortgage,"773,346","773,346","110,440","110,440","1,189,431","1,189,431","427,028","427,028","5,076,532","5,076,532",...,"1,299,364","1,299,364","259,892","259,892","1,036,422","1,036,422","101,109","101,109","318,187","318,187"
VALUE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Less than $50,000","32,992",4.30%,"2,071",1.90%,"28,652",2.40%,"18,189",4.30%,"70,874",1.40%,...,"18,772",1.40%,"14,007",5.40%,"23,805",2.30%,"3,407",3.40%,"6,289",2.00%
"$50,000 to $99,999","68,726",8.90%,"1,388",1.30%,"16,509",1.40%,"48,502",11.40%,"46,663",0.90%,...,"12,022",0.90%,"37,193",14.30%,"49,695",4.80%,"1,834",1.80%,"52,955",16.60%
"$100,000 to $299,999","404,306",52.30%,"33,203",30.10%,"225,536",19.00%,"242,819",56.90%,"272,656",5.40%,...,"115,224",8.90%,"149,478",57.50%,"538,220",51.90%,"41,709",41.30%,"227,633",71.50%


In [37]:
mortgage_data19 = pd.read_csv('../app/data/Financial_Mortgate_Data_2019.csv', index_col=0)
mortgage_data19 = mortgage_data19 .rename(columns=clean_mortgage_headers)

In [38]:
home_value = mortgage_data.iloc[[2,3,4,5,6,7,8,9]]

In [39]:
home_value19 = mortgage_data19.iloc[[2,3,4,5,6,7,8,9]]

In [40]:
def convert_value(value):
    try:
        if '%' in value:
            return float(value.replace('%', '')) / 100  # Convert percentage to a decimal
        else:
            return int(value.replace(',', ''))  # Remove commas and convert to integer
    except TypeError:
        return value  # If the value is already a float, return it as it is

In [41]:
def dataframe_sanitize(df):
    dict_mortgage = df.to_dict()
    cleaned_dict = {state: {key.strip(): convert_value(value) for key, value in data.items()} for state, data in dict_mortgage.items()}

    # Clean up dataframe columns
    final_list = []
    median_dict = {}  # Dictionary to store 'Median (dollars)' for each state
    for state, data in cleaned_dict.items():
        if 'Percent' not in state:
            state_percent = cleaned_dict.get(state + ' Percent', {})
            for value, unit in data.items():
                if value != 'Median (dollars)':
                    final_dict = {}
                    final_dict['State'] = state
                    final_dict['Value'] = value
                    final_dict['Units'] = unit
                    final_dict['unit_percent'] = state_percent.get(value)
                    final_list.append(final_dict)
                else:
                    median_dict[state] = unit  # Store 'Median (dollars)' in median_dict

    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)

    # Create a separate DataFrame for 'Median'
    median_df = pd.DataFrame(list(median_dict.items()), columns=['State', 'Median'])
    median_df['Code'] = median_df['State'].map(state_abbreviations)

    return df, median_df

In [42]:
df, df_median = dataframe_sanitize(home_value)

In [43]:
df.head()

,State,Value,Units,unit_percent,Code
0,Alabama,"Less than $50,000",32992,0.043,AL
1,Alabama,"$50,000 to $99,999",68726,0.089,AL
2,Alabama,"$100,000 to $299,999",404306,0.523,AL
3,Alabama,"$300,000 to $499,999",184159,0.238,AL
4,Alabama,"$500,000 to $749,999",56363,0.073,AL


In [44]:
df[df['State']=='Alabama']

,State,Value,Units,unit_percent,Code
0,Alabama,"Less than $50,000",32992,0.043,AL
1,Alabama,"$50,000 to $99,999",68726,0.089,AL
2,Alabama,"$100,000 to $299,999",404306,0.523,AL
3,Alabama,"$300,000 to $499,999",184159,0.238,AL
4,Alabama,"$500,000 to $749,999",56363,0.073,AL
5,Alabama,"$750,000 to $999,999",16980,0.022,AL
6,Alabama,"$1,000,000 or more",9820,0.013,AL


In [45]:
df19, df_median19 = dataframe_sanitize(home_value19)

In [46]:
df19.head()

,State,Value,Units,unit_percent,Code
0,Alabama,"Less than $50,000",37313,0.052,AL
1,Alabama,"$50,000 to $99,999",117538,0.163,AL
2,Alabama,"$100,000 to $299,999",434299,0.602,AL
3,Alabama,"$300,000 to $499,999",95611,0.132,AL
4,Alabama,"$500,000 to $749,999",22717,0.031,AL


In [47]:
def graph_bar(df, state: str, data_year: str):
    target_state = df[df['State']==state]
    # Create the bar chart
    fig = px.bar(target_state, x='Value', y='Units', color='Value', title=f'{data_year} Number of Housing Units by Cost in {state}')
    fig.update_xaxes(title_text='Home Value')
    fig.update_yaxes(title_text='Number of Houses')
    fig.show()


In [48]:
graph_bar(df,'Florida','2022')

In [49]:
def graph_bar_median_price(df, data_year: str):
    # Create the bar chart
    fig = px.bar(df, x='State', y='Median', color='State', title=f'{data_year} Median Price of Homes in US')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Median Price')
    fig.show()

In [50]:
df_median.head()

,State,Median,Code
0,Alabama,236000,AL
1,Alaska,357600,AK
2,Arizona,435200,AZ
3,Arkansas,211200,AR
4,California,738000,CA


In [51]:
graph_bar_median_price(df_median, '2022')

# Mortgage Status

In [52]:
mortgage_status = mortgage_data.iloc[[12,13,14,15,16]]

In [53]:
mortgage_status

,Alabama,Alabama Percent,Alaska,Alaska Percent,Arizona,Arizona Percent,Arkansas,Arkansas Percent,California,California Percent,...,Washington,Washington Percent,West Virginia,West Virginia Percent,Wisconsin,Wisconsin Percent,Wyoming,Wyoming Percent,Puerto Rico,Puerto Rico Percent
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Only Second mortgage,"10,065",1.30%,776,0.70%,"12,488",1.00%,"6,874",1.60%,"71,460",1.40%,...,"15,817",1.20%,"2,440",0.90%,"16,718",1.60%,"3,351",3.30%,"5,843",1.80%
Only Home equity loan,"48,242",6.20%,"3,512",3.20%,"65,377",5.50%,"16,953",4.00%,"374,777",7.40%,...,"108,322",8.30%,"16,653",6.40%,"110,921",10.70%,"4,234",4.20%,"1,966",0.60%
Both second mortgage and home equity loan,"1,629",0.20%,572,0.50%,"2,599",0.20%,559,0.10%,"18,891",0.40%,...,"5,614",0.40%,875,0.30%,"3,669",0.40%,263,0.30%,619,0.20%
No second mortgage and no home equity loan,"692,388",89.50%,"103,696",93.90%,"1,091,711",91.80%,"395,679",92.70%,"4,537,791",89.40%,...,"1,143,676",88.00%,"232,108",89.30%,"869,055",83.90%,"92,082",91.10%,"308,957",97.10%
Home equity loan without a primary mortgage,"21,022",2.70%,"1,884",1.70%,"17,256",1.50%,"6,963",1.60%,"73,613",1.50%,...,"25,935",2.00%,"7,816",3.00%,"36,059",3.50%,"1,179",1.20%,802,0.30%


In [54]:
df, df_median = dataframe_sanitize(mortgage_status)

In [55]:
df.head()

,State,Value,Units,unit_percent,Code
0,Alabama,Only Second mortgage,10065,0.013,AL
1,Alabama,Only Home equity loan,48242,0.062,AL
2,Alabama,Both second mortgage and home equity loan,1629,0.002,AL
3,Alabama,No second mortgage and no home equity loan,692388,0.895,AL
4,Alabama,Home equity loan without a primary mortgage,21022,0.027,AL


In [56]:
def graph_pie(df,state:str,data_year: str):
    target_state = df[df['State']==state]
    # Create the bar chart
    fig = px.pie(target_state, values='Units', names='Value', title=f'{data_year} Number of Houses by Mortgage Type in {state}')
    fig.show()

In [57]:
graph_pie(df,'California','2022')

# Household Income

In [58]:
household_income = mortgage_data.iloc[[18,19,20,21,22,23,24,25,26]]

In [59]:
household_income19 = mortgage_data19.iloc[[18,19,20,21,22,23,24,25,26]]

In [60]:
def dataframe_sanitize_income(df):
    
    dict_mortgage = df.to_dict()
    cleaned_dict = {state: {key.strip(): convert_value(value) for key, value in data.items()} for state, data in dict_mortgage.items()}

    #Clean up dataframe columns
    final_list = []
    median_dict = {}  # Dictionary to store 'Median household income (dollars)' for each state
    for state, data in cleaned_dict.items():
        if 'Percent' not in state:
            state_percent = cleaned_dict.get(state + ' Percent', {})
            for value, owners in data.items():
                if value != 'Median household income (dollars)':
                    final_dict = {}
                    final_dict['State'] = state
                    final_dict['Value'] = value
                    final_dict['Owners'] = owners
                    final_dict['Owner_percent'] = state_percent.get(value)
                    final_list.append(final_dict)
                else:
                    median_dict[state] = owners  # Store 'Median household income (dollars)' in median_dict
    
    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)

    # Create a separate DataFrame for 'Median'
    median_df = pd.DataFrame(list(median_dict.items()), columns=['State', 'Median'])
    median_df['Code'] = median_df['State'].map(state_abbreviations)
    

    return df, median_df

In [61]:
df, df_median = dataframe_sanitize_income(household_income)
df_median = df_median.sort_values('Median')
df.head()
df_median.head()

,State,Median,Code
51,Puerto Rico,45545,PR
24,Mississippi,81125,MS
3,Arkansas,85764,AR
48,West Virginia,87492,WV
31,New Mexico,87502,NM


In [62]:
household_income19.head()

,Alabama,Alabama Percent,Alaska,Alaska Percent,Arizona,Arizona Percent,Arkansas,Arkansas Percent,California,California Percent,...,Washington,Washington Percent,West Virginia,West Virginia Percent,Wisconsin,Wisconsin Percent,Wyoming,Wyoming Percent,Puerto Rico,Puerto Rico Percent
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
"$10,000 to $24,999","54,028",7.5%,"3,023",3.0%,"57,638",5.3%,"34,068",8.4%,"182,183",3.7%,...,"43,373",3.5%,"18,440",7.4%,"41,576",4.1%,"3,339",3.4%,"70,388",22.8%
"$25,000 to $34,999","45,500",6.3%,"2,859",2.9%,"52,718",4.8%,"30,215",7.5%,"173,591",3.5%,...,"41,719",3.3%,"17,766",7.2%,"43,882",4.4%,"3,773",3.8%,"43,349",14.1%
"$35,000 to $49,999","79,494",11.0%,"6,899",6.9%,"111,381",10.2%,"47,142",11.7%,"301,676",6.1%,...,"78,007",6.2%,"30,454",12.3%,"92,763",9.2%,"10,052",10.1%,"54,024",17.5%
"$50,000 to $74,999","137,233",19.0%,"16,169",16.3%,"193,285",17.8%,"81,579",20.2%,"627,551",12.6%,...,"183,026",14.6%,"51,179",20.7%,"184,627",18.4%,"19,679",19.8%,"54,422",17.6%
"$75,000 to $99,999","111,594",15.5%,"13,739",13.8%,"177,368",16.3%,"65,453",16.2%,"645,924",13.0%,...,"188,081",15.0%,"41,530",16.8%,"185,650",18.5%,"18,716",18.9%,"28,012",9.1%


In [63]:
df_inc_19, df_median_inc_19 = dataframe_sanitize_income(household_income19)

In [64]:
def chart_income(df, state:str,data_year:str):

    target_state = df[df['State']==state]
    fig = px.bar(target_state, x="Owners", y="Value", orientation='h', title=f'{data_year} Number of Homeowners with Household Income in {state}')
    fig.update_xaxes(title_text='Number of Homeowners')
    fig.update_yaxes(title_text='Household Income')
    fig.show()

    fig = px.pie(target_state, values='Owners', names='Value', title=f'{data_year} Percent of Homeowners by Household Income in {state}')
    fig.show()

In [65]:
chart_income(df,'Alabama', '2022')

In [66]:
def chart_income_median(df, data_year:str):
    fig = px.bar(df, x="State", y="Median", color='State', title=f'{data_year} Median Household Income by Homeowners in US')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Median Household Income')
    fig.show()
    

In [67]:
chart_income_median(df_median, '2022')

# Monthly Mortgage Payments

In [68]:
monthly_mortgage = mortgage_data.iloc[[34,35,36,37,38,39,40,41,42,43,44]]

In [69]:
monthly_mortgage

,Alabama,Alabama Percent,Alaska,Alaska Percent,Arizona,Arizona Percent,Arkansas,Arkansas Percent,California,California Percent,...,Washington,Washington Percent,West Virginia,West Virginia Percent,Wisconsin,Wisconsin Percent,Wyoming,Wyoming Percent,Puerto Rico,Puerto Rico Percent
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Less than $200,406,0.10%,0,0.00%,"1,567",0.10%,205,0.00%,"7,688",0.20%,...,755,0.10%,158,0.10%,336,0.00%,352,0.30%,479,0.20%
$200 to $399,"4,944",0.60%,742,0.70%,"7,168",0.60%,"2,377",0.60%,"16,238",0.30%,...,"3,093",0.20%,"2,197",0.80%,"2,434",0.20%,705,0.70%,"7,921",2.50%
$400 to $599,"25,252",3.30%,962,0.90%,"16,242",1.40%,"14,409",3.40%,"23,944",0.50%,...,"9,040",0.70%,"12,191",4.70%,"14,104",1.40%,"1,305",1.30%,"45,018",14.10%
$600 to $799,"70,004",9.10%,"1,237",1.10%,"48,876",4.10%,"49,388",11.60%,"43,783",0.90%,...,"18,327",1.40%,"36,062",13.90%,"49,291",4.80%,"3,646",3.60%,"77,824",24.50%
$800 to $999,"110,979",14.40%,"3,028",2.70%,"96,767",8.10%,"69,948",16.40%,"89,051",1.80%,...,"34,189",2.60%,"44,763",17.20%,"96,162",9.30%,"10,111",10.00%,"65,565",20.60%
"$1,000 to $1,499","273,252",35.30%,"17,635",16.00%,"344,732",29.00%,"151,420",35.50%,"480,514",9.50%,...,"189,769",14.60%,"92,732",35.70%,"327,949",31.60%,"30,369",30.00%,"77,087",24.20%
"$1,500 to $1,999","158,824",20.50%,"30,610",27.70%,"317,454",26.70%,"73,845",17.30%,"773,002",15.20%,...,"277,273",21.30%,"41,846",16.10%,"269,126",26.00%,"27,079",26.80%,"24,174",7.60%
"$2,000 to $2,499","65,388",8.50%,"26,015",23.60%,"168,378",14.20%,"30,828",7.20%,"845,237",16.60%,...,"258,714",19.90%,"17,065",6.60%,"138,097",13.30%,"14,714",14.60%,"8,357",2.60%
"$2,500 to $2,999","29,512",3.80%,"16,060",14.50%,"85,715",7.20%,"16,447",3.90%,"745,890",14.70%,...,"176,651",13.60%,"7,778",3.00%,"68,553",6.60%,"6,298",6.20%,"3,571",1.10%


In [70]:
df, df_median = dataframe_sanitize(monthly_mortgage)

In [71]:
df.head()

,State,Value,Units,unit_percent,Code
0,Alabama,Less than $200,406,0.001,AL
1,Alabama,$200 to $399,4944,0.006,AL
2,Alabama,$400 to $599,25252,0.033,AL
3,Alabama,$600 to $799,70004,0.091,AL
4,Alabama,$800 to $999,110979,0.144,AL


In [72]:
def graph_bar_pmt(df, state: str, data_year: str):
    target_state = df[df['State']==state]
    # Create the bar chart
    fig = px.bar(target_state, x='Value', y='Units', color='Value', title=f'{data_year} Number of Homes by Monthly Mortgage Payments in {state}')
    fig.update_xaxes(title_text='Monthly Mortgage')
    fig.update_yaxes(title_text='Number of Houses')
    fig.show()

In [73]:
graph_bar_pmt(df,'Virginia', '2022')

In [74]:
def graph_bar_pmt_median(df, data_year: str):
    # Create the bar chart
    fig = px.bar(df, x='State', y='Median', color='State', title=f'{data_year} Median Monthly Mortgage Payments in US')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Median Monthly Payment')
    fig.show()

In [75]:
graph_bar_pmt_median(df_median, '2022')

# Monthly Mortgage as percent of Income

In [76]:
#monthly_mortgage_to_income = mortgage_data.iloc[[46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66]]

# Real Estate Taxes

In [77]:
real_estate_tax = mortgage_data.iloc[[68,69,70,71,72]]

In [78]:
real_estate_tax

,Alabama,Alabama Percent,Alaska,Alaska Percent,Arizona,Arizona Percent,Arkansas,Arkansas Percent,California,California Percent,...,Washington,Washington Percent,West Virginia,West Virginia Percent,Wisconsin,Wisconsin Percent,Wyoming,Wyoming Percent,Puerto Rico,Puerto Rico Percent
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Less than $800,"327,796",42.40%,"4,405",4.00%,"138,034",11.60%,"137,458",32.20%,"139,271",2.70%,...,"45,541",3.50%,"91,389",35.20%,"25,358",2.40%,"9,807",9.70%,"35,983",11.30%
"$800 to $1,499","190,633",24.70%,"7,753",7.00%,"297,423",25.00%,"115,050",26.90%,"272,513",5.40%,...,"70,185",5.40%,"89,794",34.60%,"66,795",6.40%,"30,685",30.30%,"16,623",5.20%
"$1,500 or more","166,362",21.50%,"88,188",79.90%,"712,850",59.90%,"150,228",35.20%,"4,558,161",89.80%,...,"1,155,040",88.90%,"70,651",27.20%,"932,529",90.00%,"58,452",57.80%,"12,672",4.00%
No real estate taxes paid,"88,555",11.50%,"10,094",9.10%,"41,124",3.50%,"24,292",5.70%,"106,587",2.10%,...,"28,598",2.20%,"8,058",3.10%,"11,740",1.10%,"2,165",2.10%,"252,909",79.50%
Median (dollars),842,842,"4,039","4,039","1,873","1,873","1,183","1,183","5,479","5,479",...,"4,400","4,400","1,012","1,012","3,587","3,587","1,729","1,729",716,716


In [79]:
def dataframe_sanitize_re(df):
    
    dict_mortgage = df.to_dict()
    cleaned_dict = {state: {key.strip(): convert_value(value) for key, value in data.items()} for state, data in dict_mortgage.items()}

    #Clean up dataframe columns
    final_list = []
    median_dict = {}  # Dictionary to store 'Median (dollars)' for each state
    for state, data in cleaned_dict.items():
        if 'Percent' not in state:
            state_percent = cleaned_dict.get(state + ' Percent', {})
            for value, unit in data.items():
                if value != 'Median (dollars)':
                    final_dict = {}
                    final_dict['State'] = state
                    final_dict['Tax'] = value
                    final_dict['Units'] = unit
                    final_dict['unit_percent'] = state_percent.get(value)
                    final_list.append(final_dict)
                else:
                    median_dict[state] = unit  # Store 'Median (dollars)' in median_dict
    
    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)

    # Create a separate DataFrame for 'Median'
    median_df = pd.DataFrame(list(median_dict.items()), columns=['State', 'Median'])
    median_df['Code'] = median_df['State'].map(state_abbreviations)

    return df, median_df

In [80]:
df, df_median = dataframe_sanitize_re(real_estate_tax)

In [81]:
df.head()

,State,Tax,Units,unit_percent,Code
0,Alabama,Less than $800,327796,0.424,AL
1,Alabama,"$800 to $1,499",190633,0.247,AL
2,Alabama,"$1,500 or more",166362,0.215,AL
3,Alabama,No real estate taxes paid,88555,0.115,AL
4,Alaska,Less than $800,4405,0.040,AK


In [82]:
def graph_pie_tax(df,state:str,data_year: str):
    target_state = df[df['State']==state]
    # Create the bar chart
    fig = px.pie(target_state, values='Units', names='Tax', title=f'{data_year} Real Estate Taxes Paid by Household in {state}')
    fig.show()

In [83]:
graph_pie_tax(df,'Virginia', '2022')

In [84]:
df_median.head()

,State,Median,Code
0,Alabama,842,AL
1,Alaska,4039,AK
2,Arizona,1873,AZ
3,Arkansas,1183,AR
4,California,5479,CA


# Owner occupied Housing Units

In [85]:
housing_units = mortgage_data.iloc[0]

In [86]:
housing_units

Alabama                  773,346
Alabama Percent          773,346
Alaska                   110,440
Alaska Percent           110,440
Arizona                1,189,431
                         ...    
Wisconsin Percent      1,036,422
Wyoming                  101,109
Wyoming Percent          101,109
Puerto Rico              318,187
Puerto Rico Percent      318,187
Name: Owner-occupied housing units with a mortgage, Length: 104, dtype: object

In [87]:
def clean_units_df(df):
    dict_units = df.to_dict()
    final_list = []
    for state, data in dict_units.items():
        if 'Percent' not in state:
            final_dict = {}
            final_dict['State'] = state
            final_dict['Units'] = convert_value(data)
            final_list.append(final_dict)
    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)
    
    return df

In [88]:
df = clean_units_df(housing_units)

In [89]:
df.head()

,State,Units,Code
0,Alabama,773346,AL
1,Alaska,110440,AK
2,Arizona,1189431,AZ
3,Arkansas,427028,AR
4,California,5076532,CA


In [90]:
def chart_units(df, data_year:str):

    fig = px.bar(df, x="State", y="Units", title=f'{data_year} Number of Housing Units With A Mortgage')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Number of Housing Units')
    fig.show()

In [91]:
chart_units(df,'2022')

In [92]:
home_val, median_home =  dataframe_sanitize(home_value)

median_home.rename(columns={'Median': 'Median_House_Price'}, inplace=True)
median_home.head()

,State,Median_House_Price,Code
0,Alabama,236000,AL
1,Alaska,357600,AK
2,Arizona,435200,AZ
3,Arkansas,211200,AR
4,California,738000,CA


In [93]:
hh_income, median_hh_income = dataframe_sanitize_income(household_income)
median_hh_income.rename(columns={'Median': 'Median_Income'}, inplace=True)
median_hh_income.head()

,State,Median_Income,Code
0,Alabama,91415,AL
1,Alaska,118096,AK
2,Arizona,102270,AZ
3,Arkansas,85764,AR
4,California,134629,CA


In [94]:
house_aff = pd.merge(median_home,median_hh_income,on='State')
house_aff['Ratio'] = house_aff['Median_House_Price']/house_aff['Median_Income']
house_aff.head()

,State,Median_House_Price,Code_x,Median_Income,Code_y,Ratio
0,Alabama,236000,AL,91415,AL,2.581633
1,Alaska,357600,AK,118096,AK,3.028045
2,Arizona,435200,AZ,102270,AZ,4.255402
3,Arkansas,211200,AR,85764,AR,2.462572
4,California,738000,CA,134629,CA,5.481731


In [95]:
def chart_income_median(df, data_year:str):
    fig = px.bar(df, x="State", y="Ratio", color='State', title=f'{data_year} Home Affordability by State')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Affordability Ratio')
    fig.show()

In [96]:
house_aff = house_aff.sort_values('Ratio')
house_aff['Code'] = house_aff['State'].map(state_abbreviations)

In [97]:
house_aff.head()

,State,Median_House_Price,Code_x,Median_Income,Code_y,Ratio,Code
48,West Virginia,186900,WV,87492,WV,2.136195,WV
15,Iowa,208900,IA,97147,IA,2.150349,IA
35,Ohio,223700,OH,98122,OH,2.279815,OH
16,Kansas,234500,KS,100847,KS,2.325305,KS
27,Nebraska,248500,NE,104908,NE,2.368742,NE


In [98]:
chart_income_median(house_aff, '2022')

In [99]:

fig = px.histogram(house_aff, x="Ratio", nbins=20, color="State", title="Histogram of Ratio by State")
fig.show()



In [141]:
def make_map(df, data_year: str):

    data = dict(type = 'choropleth',
                colorscale = 'Hot_r',
                locations = df['Code'],
                locationmode = 'USA-states',
                z=df['Ratio'],
                colorbar = {'title':'Affordability Ratio'})

    layout = dict(
        title = {'text': f'{data_year} House Affordability by State', 'x':0.5, 'xanchor': 'center'},
        geo = dict(scope = 'usa'),
        autosize = True,
        width = 800,
        height = 600,
    )

    choromap = go.Figure(data = [data],layout = layout)

    iplot(choromap)

make_map(house_aff, '2022')

In [101]:
tax, tax_median = dataframe_sanitize_re(real_estate_tax)

In [102]:
tax_median.rename(columns={'Median': 'Median_Tax'}, inplace=True)
tax_median.head()

,State,Median_Tax,Code
0,Alabama,842,AL
1,Alaska,4039,AK
2,Arizona,1873,AZ
3,Arkansas,1183,AR
4,California,5479,CA


In [103]:
tax_burden = pd.merge(tax_median,median_hh_income,on='State')
tax_burden['Ratio'] = tax_burden['Median_Tax']/tax_burden['Median_Income']
tax_burden = tax_burden.sort_values('Ratio')
tax_burden['Code'] = tax_burden['State'].map(state_abbreviations)
tax_burden.head()

,State,Median_Tax,Code_x,Median_Income,Code_y,Ratio,Code
0,Alabama,842,AL,91415,AL,0.009211,AL
48,West Virginia,1012,WV,87492,WV,0.011567,WV
3,Arkansas,1183,AR,85764,AR,0.013794,AR
18,Louisiana,1396,LA,94320,LA,0.014801,LA
40,South Carolina,1363,SC,91731,SC,0.014859,SC


In [104]:
def chart_tax_median(df, data_year:str):
    fig = px.bar(df, x="State", y="Ratio", color='State', title=f'{data_year} Real Estate Tax Burden by State')
    fig.update_xaxes(title_text='State')
    fig.update_yaxes(title_text='Tax Burden Ratio')
    fig.show()

chart_tax_median(tax_burden, '2022')


# Household income accross the states

In [105]:
data_hh_2022 = pd.read_csv('../app/data/Household_income2022.csv', index_col=0)
data_hh_2022 = data_hh_2022.rename(columns=clean_headers)
data_hh_2022.head()

,Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,District of Columbia,Florida,...,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming,Puerto Rico
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Total:,"2,016,448","274,574","2,850,377","1,216,207","13,550,586","2,384,584","1,433,635","402,334","326,970","8,826,394",...,"2,846,684","11,087,708","1,129,660","277,090","3,380,607","3,079,953","736,341","2,491,121","243,321","1,289,311"
"Less than $10,000","152,705","12,191","153,140","88,467","654,370","98,418","72,746","16,288","22,335","495,572",...,"154,793","611,011","33,488","11,050","165,315","139,331","52,185","115,058","12,728","288,952"
"$10,000 to $14,999","105,647","6,898","85,230","71,588","436,122","59,498","45,957","10,537","9,163","312,253",...,"122,133","400,408","27,309","10,438","106,917","87,195","46,671","86,287","8,218","138,823"
"$15,000 to $19,999","91,762","6,734","87,775","56,753","353,371","62,866","35,686","12,008","9,690","300,563",...,"106,466","342,749","20,350","8,415","92,463","67,313","34,443","81,398","9,329","131,541"
"$20,000 to $24,999","96,250","8,104","88,905","58,233","395,762","64,982","48,394","11,328","7,068","336,366",...,"114,587","392,533","26,116","10,243","103,515","84,023","43,006","93,460","9,013","103,114"


In [106]:
def dataframe_clean_hh(df):
    dict_mortgage = df.to_dict()
    cleaned_dict = {state: {key.strip(): convert_value(value) for key, value in data.items()} for state, data in dict_mortgage.items()}

    # Clean up dataframe columns
    final_list = []
    total_units = {}
    for state, data in cleaned_dict.items():
        for income, unit in data.items():
            if income != 'Total:':
                final_dict = {}
                final_dict['State'] = state
                final_dict['Income'] = income
                final_dict['Units'] = unit
                final_list.append(final_dict)
            else:
                total_units[state] = unit


    df = pd.DataFrame(final_list)
    df['Code'] = df['State'].map(state_abbreviations)

    df_total = pd.DataFrame(list(total_units.items()), columns=['State', 'Units'])
    df_total['Code'] = df_total['State'].map(state_abbreviations)

    return df, df_total

In [107]:
total_hh_income, total_state_units = dataframe_clean_hh(data_hh_2022)

In [108]:
total_hh_income.head()

,State,Income,Units,Code
0,Alabama,"Less than $10,000",152705,AL
1,Alabama,"$10,000 to $14,999",105647,AL
2,Alabama,"$15,000 to $19,999",91762,AL
3,Alabama,"$20,000 to $24,999",96250,AL
4,Alabama,"$25,000 to $29,999",87477,AL


In [109]:
total_state_units = total_state_units.sort_values('Units')
total_state_units.head()

,State,Units,Code
50,Wyoming,243321,WY
1,Alaska,274574,AK
45,Vermont,277090,VT
8,District of Columbia,326970,DC
34,North Dakota,331481,ND


In [110]:
def chart_income_total(df, state:str,data_year:str):

    target_state = df[df['State']==state]
    fig = px.bar(target_state, x="Units", y="Income", orientation='h', title=f'{data_year} Number of Household Units with Income in {state}')
    fig.update_xaxes(title_text='Number of Household Units')
    fig.update_yaxes(title_text='Household Income')
    fig.show()

In [111]:
chart_income_total(total_hh_income,'Texas', '2022')

# Physical House Occupancy Characteristics

In [112]:
def clean_house_char_headers(val):
    if isinstance(val, str):
        if 'Occupied' in val:
            val = val.split("!!")[0]
            val = val + "_total"
        elif 'Percent occupied housing units' in val:
            val = val.split("!!")[0]
            val = val + "_total_percent"
        elif 'Owner-occupied housing'in val:
            val = val.split("!!")[0]
            val = val + "_owner"
        elif 'Percent owner-occupied housing units' in val:
            val = val.split("!!")[0]
            val = val + "_own_percent"
        elif 'Renter-occupied housing units' in val:
            val = val.split("!!")[0]
            val = val + "_renter"
        elif 'Percent renter-occupied' in val:
            val = val.split("!!")[0]
            val = val + "_rent_percent"
        else:
            val = val.split("!!")[0]
        return val
    else:
        return val

In [113]:
house_char_data = pd.read_csv('../app/data/Physical_Housing_Occup.csv', index_col=0)
house_char_data = house_char_data.rename(columns=clean_house_char_headers)
house_char_data.head()

,Alabama_total,Alabama_total_percent,Alabama_owner,Alabama_own_percent,Alabama_renter,Alabama_rent_percent,Alaska_total,Alaska_total_percent,Alaska_owner,Alaska_own_percent,...,Wyoming_owner,Wyoming_own_percent,Wyoming_renter,Wyoming_rent_percent,Puerto Rico_total,Puerto Rico_total_percent,Puerto Rico_owner,Puerto Rico_own_percent,Puerto Rico_renter,Puerto Rico_rent_percent
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
Occupied housing units,"2,016,448","2,016,448","1,416,333","1,416,333","600,115","600,115","274,574","274,574","181,586","181,586",...,"176,863","176,863","66,458","66,458","1,289,311","1,289,311","869,635","869,635","419,676","419,676"
UNITS IN STRUCTURE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"1, detached","1,436,137",71.2%,"1,205,520",85.1%,"230,617",38.4%,"170,997",62.3%,"149,053",82.1%,...,"147,580",83.4%,"20,001",30.1%,"890,441",69.1%,"703,191",80.9%,"187,250",44.6%
"1, attached","41,268",2.0%,"23,036",1.6%,"18,232",3.0%,"22,604",8.2%,"13,565",7.5%,...,"7,390",4.2%,"4,919",7.4%,"145,191",11.3%,"83,088",9.6%,"62,103",14.8%
2 apartments,"35,683",1.8%,"1,255",0.1%,"34,428",5.7%,"15,026",5.5%,"4,122",2.3%,...,873,0.5%,"3,457",5.2%,"34,016",2.6%,"14,680",1.7%,"19,336",4.6%


In [114]:
units_in_struc = house_char_data.iloc[[2,3,4,5,6,7,8]]
units_in_struc.head()

,Alabama_total,Alabama_total_percent,Alabama_owner,Alabama_own_percent,Alabama_renter,Alabama_rent_percent,Alaska_total,Alaska_total_percent,Alaska_owner,Alaska_own_percent,...,Wyoming_owner,Wyoming_own_percent,Wyoming_renter,Wyoming_rent_percent,Puerto Rico_total,Puerto Rico_total_percent,Puerto Rico_owner,Puerto Rico_own_percent,Puerto Rico_renter,Puerto Rico_rent_percent
Label (Grouping),,,,,,,,,,,,,,,,,,,,,
"1, detached","1,436,137",71.2%,"1,205,520",85.1%,"230,617",38.4%,"170,997",62.3%,"149,053",82.1%,...,"147,580",83.4%,"20,001",30.1%,"890,441",69.1%,"703,191",80.9%,"187,250",44.6%
"1, attached","41,268",2.0%,"23,036",1.6%,"18,232",3.0%,"22,604",8.2%,"13,565",7.5%,...,"7,390",4.2%,"4,919",7.4%,"145,191",11.3%,"83,088",9.6%,"62,103",14.8%
2 apartments,"35,683",1.8%,"1,255",0.1%,"34,428",5.7%,"15,026",5.5%,"4,122",2.3%,...,873,0.5%,"3,457",5.2%,"34,016",2.6%,"14,680",1.7%,"19,336",4.6%
3 or 4 apartments,"57,324",2.8%,"2,986",0.2%,"54,338",9.1%,"20,093",7.3%,"2,783",1.5%,...,198,0.1%,"10,707",16.1%,"33,814",2.6%,"7,645",0.9%,"26,169",6.2%
5 to 9 apartments,"75,649",3.8%,"2,717",0.2%,"72,932",12.2%,"15,293",5.6%,"2,599",1.4%,...,384,0.2%,"7,442",11.2%,"59,961",4.7%,"16,203",1.9%,"43,758",10.4%
